In [49]:
import random
import math
import datetime

In [6]:
arrival_probabilities = [
    0.0094,
    0.0094,
    0.0094,
    0.0094,
    0.0094,
    0.0094,
    0.0094,
    0.0094,
    0.0283,
    0.0283,
    0.0566,
    0.0566,
    0.0566,
    0.0755,
    0.0755,
    0.0755,
    0.1038,
    0.1038,
    0.1038,
    0.0472,
    0.0472,
    0.0472,
    0.0094,
    0.0094
]

charging_demand_probabilities = [
    (0.3431, 0),
    (0.0490, 5),
    (0.0980, 10),
    (0.1176, 20),
    (0.0882, 30),
    (0.1176, 50),
    (0.1078, 100),
    (0.0490, 200),
    (0.0294, 300)
]

In [41]:
def simulate_chargepoint(ticks_per_year, ticks_per_hour, chargepoint_states, total_energy_per_tick, power_demand_per_tick, charging_power, minutes_per_tick, kwh_per_100km):
    for tick in range(ticks_per_year):
        if chargepoint_states[tick] is False:
            hour_of_day = (tick // ticks_per_hour) % 24
            if random.random() < arrival_probabilities[hour_of_day]:
                charging_demand = random.choices(charging_demand_probabilities, weights=[p[0] for p in charging_demand_probabilities])[0][1]
                if charging_demand > 0:
                    charge_ticks_remaining = int(charging_demand / 100 * kwh_per_100km / charging_power * 60 / minutes_per_tick)
                    for i in range(tick, min(tick + charge_ticks_remaining, ticks_per_year)):
                        chargepoint_states[i] = True
                        total_energy_per_tick[i] += charging_power * minutes_per_tick / 60
                        power_demand_per_tick[i] += charging_power
        else:
            chargepoint_states[tick] = False

In [45]:
def run(num_chargepoints, charging_power, minutes_per_tick, kwh_per_100km, simulation_fn = simulate_chargepoint):
    ticks_per_hour = 60 // minutes_per_tick
    ticks_per_day = 24 * ticks_per_hour
    ticks_per_year = 365 * ticks_per_day

    chargepoint_states = {tick: False for tick in range(ticks_per_year)}
    total_energy_per_tick = {tick: 0 for tick in range(ticks_per_year)}
    power_demand_per_tick = {tick: 0 for tick in range(ticks_per_year)}

    for _ in range(num_chargepoints):
        simulate_chargepoint(ticks_per_year, ticks_per_hour, chargepoint_states, total_energy_per_tick, power_demand_per_tick, charging_power, minutes_per_tick, kwh_per_100km)

    total_energy_consumed = sum(total_energy_per_tick.values())
    max_power_demand = max(power_demand_per_tick.values())
    theoretical_max_power_demand = num_chargepoints * charging_power

    return {
        'total_energy_consumed': total_energy_consumed,
        'max_power_demand': max_power_demand,
        'theoretical_max_power_demand': theoretical_max_power_demand,
        'concurrency_factor': max_power_demand / theoretical_max_power_demand
    }

In [46]:
NUM_CHARGEPOINTS = 20
CHARGING_POWER = 11  # kW
MINUTES_PER_TICK = 15
KWH_PER_100KM = 18

output = run(NUM_CHARGEPOINTS, CHARGING_POWER, MINUTES_PER_TICK, KWH_PER_100KM)

print(f"Total energy consumed: {output['total_energy_consumed']:.2f} kWh")
print(f"Theoretical maximum power demand: {output['theoretical_max_power_demand']:.2f} kW")
print(f"Actual maximum power demand: {output['max_power_demand']:.2f} kW")
print(f"Concurrency factor: {output['concurrency_factor']:.2f}")

Total energy consumed: 171171.00 kWh
Theoretical maximum power demand: 220.00 kW
Actual maximum power demand: 121.00 kW
Concurrency factor: 0.55


In [44]:
# Run the program from task 1 for between 1 and 30 chargepoints. How does the concurrency factor behave?

MAX_NUM_CHARGEPOINTS = 30
CHARGING_POWER = 11  # kW
MINUTES_PER_TICK = 15
KWH_PER_100KM = 18

concurrency_factors = []

for num_chargepoints in range(1, MAX_NUM_CHARGEPOINTS + 1):
    output = run(num_chargepoints, CHARGING_POWER, MINUTES_PER_TICK, KWH_PER_100KM)
    factor = output['concurrency_factor']
    print(f"{num_chargepoints} chargepoint(s) - factor: {factor:.2f} - deviation from 35%: {factor - 0.35:.2f} - deviation from 55%: {factor - 0.55:.2f}")
    concurrency_factors.append(factor)

# Calculate standard deviation
mean = sum(concurrency_factors) / len(concurrency_factors)
variances = [(factor - mean) ** 2 for factor in concurrency_factors]
standard_deviation = math.sqrt(sum(variances) / len(variances))

print(f"\nMean Concurrency Factor: {mean:.2f}")
print(f"Standard Deviation: {standard_deviation:.2f}")

1 chargepoint(s) - factor: 1.00 - deviation from 35%: 0.65 - deviation from 55%: 0.45
2 chargepoint(s) - factor: 1.00 - deviation from 35%: 0.65 - deviation from 55%: 0.45
3 chargepoint(s) - factor: 1.00 - deviation from 35%: 0.65 - deviation from 55%: 0.45
4 chargepoint(s) - factor: 1.00 - deviation from 35%: 0.65 - deviation from 55%: 0.45
5 chargepoint(s) - factor: 0.80 - deviation from 35%: 0.45 - deviation from 55%: 0.25
6 chargepoint(s) - factor: 0.83 - deviation from 35%: 0.48 - deviation from 55%: 0.28
7 chargepoint(s) - factor: 0.86 - deviation from 35%: 0.51 - deviation from 55%: 0.31
8 chargepoint(s) - factor: 0.75 - deviation from 35%: 0.40 - deviation from 55%: 0.20
9 chargepoint(s) - factor: 0.67 - deviation from 35%: 0.32 - deviation from 55%: 0.12
10 chargepoint(s) - factor: 0.70 - deviation from 35%: 0.35 - deviation from 55%: 0.15
11 chargepoint(s) - factor: 0.73 - deviation from 35%: 0.38 - deviation from 55%: 0.18
12 chargepoint(s) - factor: 0.67 - deviation from 35

In [51]:
# If you consider the impact of DST vs. mapping the hours to the 15 minute ticks.

DST_START = datetime.datetime(2024, 3, 10)  # March 10, 2024
DST_END = datetime.datetime(2024, 11, 3)    # November 3, 2024

def simulate_chargepoint_with_DTS(ticks_per_year, ticks_per_hour, chargepoint_states, total_energy_per_tick, power_demand_per_tick, charging_power, minutes_per_tick, kwh_per_100km):    
    for tick in range(ticks_per_year):
        dts_tick = tick
        
        date = datetime.datetime(2024, 1, 1) + datetime.timedelta(minutes=tick * minutes_per_tick)
        if DST_START <= date < DST_END:
            # During DST period, the hour of the day is shifted by 1
            dts_tick = tick - ticks_per_hour
        
        if chargepoint_states[tick] is False:
            hour_of_day = (dts_tick // ticks_per_hour) % 24
            if random.random() < arrival_probabilities[hour_of_day]:
                charging_demand = random.choices(charging_demand_probabilities, weights=[p[0] for p in charging_demand_probabilities])[0][1]
                if charging_demand > 0:
                    charge_ticks_remaining = int(charging_demand / 100 * kwh_per_100km / charging_power * 60 / minutes_per_tick)
                    for i in range(tick, min(tick + charge_ticks_remaining, ticks_per_year)):
                        chargepoint_states[i] = True
                        total_energy_per_tick[i] += charging_power * minutes_per_tick / 60
                        power_demand_per_tick[i] += charging_power
        else:
            chargepoint_states[tick] = False

NUM_CHARGEPOINTS = 20
CHARGING_POWER = 11  # kW
MINUTES_PER_TICK = 15
KWH_PER_100KM = 18

output = run(NUM_CHARGEPOINTS, CHARGING_POWER, MINUTES_PER_TICK, KWH_PER_100KM, simulate_chargepoint_with_DTS)

print(f"Total energy consumed: {output['total_energy_consumed']:.2f} kWh")
print(f"Theoretical maximum power demand: {output['theoretical_max_power_demand']:.2f} kW")
print(f"Actual maximum power demand: {output['max_power_demand']:.2f} kW")
print(f"Concurrency factor: {output['concurrency_factor']:.2f}")

Total energy consumed: 167216.50 kWh
Theoretical maximum power demand: 220.00 kW
Actual maximum power demand: 110.00 kW
Concurrency factor: 0.50
